# Top 10 Airports

Using `bookings` file.

* Arrival airport is the column arr_port. It is the IATA code for the airport

* To get the **total number of passengers** for an airport, you can **sum the column
pax**, **grouping by arr_port**.
Note that there is negative pax. That corresponds to
cancelations. So to get the total number of passengers that have actually
booked, you should sum including the negatives (that will remove the canceled
bookings).
* **Print the top 10 arrival airports** in the standard output, **including the number of
passengers**.
* Bonus point: Get the name of the city or airport corresponding to that airport
(programatically, we suggest to have a look at GeoBases in Github)
* Bonus point: Solve this problem using pandas (instead of any other approach)

## Load modules and libraries

In [1]:
import pandas as pd

## File load

Here the `pd.read_csv` help to fully explain what we're doing when loading the file.

[Docs pandas.read_csv](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)

In [ ]:
help(pd.read_csv)

In [7]:
file_path = "/home/miki/Documents/data/challenge/bookings.csv.bz2"
df_bookings = pd.read_csv(
    file_path,
    compression = 'bz2',    # we know it's compressed although read_csv can infer the compression type
    sep = '^',        # Weird separator used
    nrows = 100,            # start small to go big
)

## Basic exploration

In [8]:
df_bookings.columns

Index(['act_date           ', 'source', 'pos_ctry', 'pos_iata', 'pos_oid  ',
       'rloc          ', 'cre_date           ', 'duration', 'distance',
       'dep_port', 'dep_city', 'dep_ctry', 'arr_port', 'arr_city', 'arr_ctry',
       'lst_port', 'lst_city', 'lst_ctry', 'brd_port', 'brd_city', 'brd_ctry',
       'off_port', 'off_city', 'off_ctry', 'mkt_port', 'mkt_city', 'mkt_ctry',
       'intl', 'route          ', 'carrier', 'bkg_class', 'cab_class',
       'brd_time           ', 'off_time           ', 'pax', 'year', 'month',
       'oid      '],
      dtype='object')

We have malformed column names with extra spaces.

In [9]:
df_bookings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 38 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   act_date             100 non-null    object
 1   source               100 non-null    object
 2   pos_ctry             100 non-null    object
 3   pos_iata             100 non-null    object
 4   pos_oid              100 non-null    object
 5   rloc                 100 non-null    object
 6   cre_date             100 non-null    object
 7   duration             100 non-null    int64 
 8   distance             100 non-null    int64 
 9   dep_port             100 non-null    object
 10  dep_city             100 non-null    object
 11  dep_ctry             100 non-null    object
 12  arr_port             100 non-null    object
 13  arr_city             100 non-null    object
 14  arr_ctry             100 non-null    object
 15  lst_port             100 non-null    object
 16  lst_city 

Too much columns, to solve the problem we only need:
* **arr_port**, arrival airpot. *Column 12*
* **pax**, number of passengers. *Column 34*
* **year**, year of the measurement. *Column 35*

In [40]:
cols_of_interest = [12, 34, 35]
df_bookings = pd.read_csv(
    file_path,
    compression="bz2",  # We know it's compressed although read_csv can infer the compression type
    sep="^",  # Weird separator used
    nrows=100,  # start small to go big
    usecols= cols_of_interest  # Can select columns by name (not ideal in this case) or by position
)
df_bookings.head()


,arr_port,pax,year
0,LHR,-1,2013
1,CLT,1,2013
2,CLT,1,2013
3,SVO,1,2013
4,SVO,1,2013


In [22]:
# Cleanup column names, just in case
df_bookings.columns = df_bookings.columns.str.strip()
df_bookings.columns

Index(['arr_port', 'pax', 'year'], dtype='object')

In [18]:
arr_port_values = df_bookings["arr_port"].unique()
print(arr_port_values)


['LHR     ' 'CLT     ' 'SVO     ' 'LGA     ' 'SIN     ' 'TUS     '
 'CTA     ' 'YWG     ' 'PVG     ' 'SGC     ' 'ALG     ' 'DMM     '
 'LED     ' 'LIS     ' 'BLR     ' 'BWI     ' 'DEL     ' 'TPA     '
 'MNL     ' 'ULN     ' 'ZRH     ' 'ALA     ' 'FCO     ' 'AYT     '
 'MVD     ' 'CVG     ' 'OTP     ' 'CDG     ' 'HAV     ' 'NRT     ']


Airport IATA codes have extra spaces.

In [27]:
df_bookings["arr_port"] = df_bookings["arr_port"].str.strip()
df_bookings["arr_port"].unique()

array(['LHR', 'CLT', 'SVO', 'LGA', 'SIN', 'TUS', 'CTA', 'YWG', 'PVG',
       'SGC', 'ALG', 'DMM', 'LED', 'LIS', 'BLR', 'BWI', 'DEL', 'TPA',
       'MNL', 'ULN', 'ZRH', 'ALA', 'FCO', 'AYT', 'MVD', 'CVG', 'OTP',
       'CDG', 'HAV', 'NRT'], dtype=object)

`pax` and `year` columns are numeric. I should have a binned histogram, max, min, std, mean, median to see how columns are distributed but sinze we're working with only 100 rows it does not make much sense. So skipping it for now.

In [28]:
df_bookings.head()

,arr_port,pax,year
0,LHR,-1,2013
1,CLT,1,2013
2,CLT,1,2013
3,SVO,1,2013
4,SVO,1,2013


In [39]:
pax_by_airport = df_bookings.groupby(by=["year", "arr_port"])['pax'].sum()
top10 = pax_by_airport.sort_values(ascending=False).head(10)
top10

year  arr_port
2013  BWI         6
      PVG         6
      NRT         6
      YWG         4
      SIN         4
      LIS         4
      AYT         3
      FCO         3
      ALA         2
      TPA         2
Name: pax, dtype: int64

## Work with the entire dataset

The plan is:
1. Tear the dataset down to smaller pieces.
2. Perform aggregation operations on the small piece.
3. Save it for later to combine all the smaller pieces.
4. Aggregate again but with the smaller pieces or chunks of data.

We will use the `chunk` parameter of the `pd.read_csv` function.

In [43]:
cols_of_interest = [12, 34, 35]
with pd.read_csv(
    file_path,
    compression="bz2",
    sep="^",
    usecols=cols_of_interest,
    chunksize =1000
) as reader:
    reader
    for chunk in reader:
        